In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values


In [2]:
X = dataset[1:,23:30]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')
print("X:  %s, Y: %s" % (X.shape, Y.shape))
d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,3,5], axis=1, inplace=True)
d2.replace(0., np.nan, inplace=True)
d2.replace(-1., np.nan, inplace=True)
d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]
X.shape, Y.shape

X:  (918, 7), Y: (918,)


((908, 4), (908, 1))

In [3]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)

X_scale.shape, Y_scale.shape

((908, 4), (908, 1))

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


((726, 4), (726, 1), (182, 4), (182, 1))

In [5]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model


def basic_model_3(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(80, activation="tanh", kernel_initializer='normal', input_shape=(x_size,)))
    t_model.add(Dropout(0.2))
    t_model.add(Dense(120, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(20, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(10, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(5, activation="relu", kernel_initializer='normal'))
    t_model.add(Dropout(0.0))
    t_model.add(Dense(y_size))
    t_model.compile(
        loss='mean_squared_error',
        optimizer='adam',
        metrics=[metrics.mae])
    return(t_model)

model = basic_model_3(4, 1)
model.summary()

Using TensorFlow backend.
W0917 11:00:49.073679 15064 deprecation_wrapper.py:119] From C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0917 11:00:49.104676 15064 deprecation_wrapper.py:119] From C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0917 11:00:49.108706 15064 deprecation_wrapper.py:119] From C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0917 11:00:49.126682 15064 deprecation_wrapper.py:119] From C:\Users\Alecs\Anaconda3\envs\CChirita\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.p

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               9720      
_________________________________________________________________
dropout_2 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                2420      
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                210       
__________

In [6]:
epochs = 5000
batch_size = 8

print('Epochs: ', epochs)
print('Batch size: ', batch_size)

keras_callbacks = [
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
    # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
    EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
]

history = model.fit(X_train, Y_train,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=0, # Change it to 2, if wished to observe execution
    validation_data=(X_test, Y_test),
    callbacks=keras_callbacks)
 

Epochs:  5000
Batch size:  8


In [7]:
train_score = model.evaluate(X_train, Y_train, verbose=0)
valid_score = model.evaluate(X_test, Y_test, verbose=0)

print('Train MAE: ', round(train_score[1], 4), ', Train Loss: ', round(train_score[0], 4)) 
print('Val MAE: ', round(valid_score[1], 4), ', Val Loss: ', round(valid_score[0], 4))

Train MAE:  0.1788 , Train Loss:  0.0628
Val MAE:  0.1725 , Val Loss:  0.0601


In [8]:
import matplotlib.pyplot as plt
def plot_hist(h, xsize=6, ysize=10):
    # Prepare plotting
    fig_size = plt.rcParams["figure.figsize"]
    plt.rcParams["figure.figsize"] = [xsize, ysize]
    fig, axes = plt.subplots(nrows=4, ncols=4, sharex=True)
    
    # summarize history for MAE
    plt.subplot(211)
    plt.plot(h['mean_absolute_error'])
    plt.plot(h['val_mean_absolute_error'])
    plt.title('Training vs Validation MAE')
    plt.ylabel('MAE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # summarize history for loss
    plt.subplot(212)
    plt.plot(h['loss'])
    plt.plot(h['val_loss'])
    plt.title('Training vs Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot it all in IPython (non-interactive)
    plt.draw()
    plt.show()

    return

p1 = plot_hist(history.history, xsize=8, ysize=12)
plt.show()

<Figure size 800x1200 with 2 Axes>

In [9]:
emax = 1.
emin = 1.
for i in range(10):
    p = model.predict_on_batch(X_train[i].reshape(1, 4))
    pp = Y_scaler.inverse_transform(p)
    py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
    r = p/float(Y_train[i])
    if r > emax:
        emax = r
    if r < emin:
        emin = r
    print(pp,py, r)


[[10.02909]] [[7.84]] [[1.3200424]]
[[10.02909]] [[4.]] [[3.0096965]]
[[10.02909]] [[8.87]] [[1.1472795]]
[[10.02909]] [[10.26]] [[0.9750637]]
[[10.02909]] [[12.]] [[0.8208263]]
[[10.02909]] [[11.86]] [[0.8314079]]
[[10.02909]] [[6.13]] [[1.7600565]]
[[10.02909]] [[9.58]] [[1.0523415]]
[[10.02909]] [[11.79]] [[0.8368016]]
[[10.02909]] [[7.6]] [[1.3680439]]
